This notebook tests how belivable the data generated by the selfexperimentation model is.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy
from adaptive_nof1.models import Model, SelfExperimentationModel
from adaptive_nof1.basic_types import Outcome

from adaptive_nof1 import SeriesOfSimulationsRunner
from adaptive_nof1.policies import (
    FixedPolicy,
    ThompsonSampling,
    BalancedThompsonSampling,
)
from adaptive_nof1.inference import FastNormalApproximation

In [ ]:
# Simulation constants
data_generating_model = lambda patient_id: SelfExperimentationModel(
    patient_id,
    baseline_drift=True,
    baseline_correlation=0.99,
    baseline_variance=0.005,
    flare_probability=1.0 / 7,
    intervention_effects=[0, 1],
)

length = 100
number_of_actions = 2
number_of_patients = 100

Now, we create two series of simulations objects. We can use predefined policies.

In [ ]:
fixed_runner = SeriesOfSimulationsRunner(
    model_from_patient_id=data_generating_model,
    n_patients=number_of_patients,
    policy=FixedPolicy(number_of_actions=number_of_actions),
)
fixed_data = fixed_runner.simulate(length)

In [ ]:
ts_runner = SeriesOfSimulationsRunner(
    model_from_patient_id=data_generating_model,
    n_patients=number_of_patients,
    policy=ThompsonSampling(
        inference_model=FastNormalApproximation(),
        number_of_actions=number_of_actions,
    ),
)
ts_data = ts_runner.simulate(length)

In [ ]:
balanced_ts_runner = SeriesOfSimulationsRunner(
    model_from_patient_id=data_generating_model,
    n_patients=number_of_patients,
    policy=BalancedThompsonSampling(
        inference_model=FastNormalApproximation(),
        number_of_actions=number_of_actions,
    ),
)
balanced_ts_data = balanced_ts_runner.simulate(length)

In [ ]:
from adaptive_nof1.metrics import SimpleRegret, GetValue
from adaptive_nof1.series_of_simulations_data import SeriesOfSimulationsData


SeriesOfSimulationsData.plot_lines(
    [fixed_data, ts_data],
    [GetValue(outcome_name="baseline"), GetValue(outcome_name="outcome")],
    legend_position=(1, 1),
)

In [ ]:
SeriesOfSimulationsData.plot_lines(
    [fixed_data, ts_data, balanced_ts_data],
    [SimpleRegret(outcome_name="outcome")],
    legend_position=(1, 1),
)

In [ ]:
fixed_data.plot_allocations()

In [ ]:
balanced_ts_data.plot_allocations()

In [ ]:
ts_data.plot_allocations()